In [1]:
import pdfplumber
import os
import re
import pandas as pd
import sys
import datetime
import calendar
from tqdm import tqdm
import win32com
import win32com.client
import docx

In [2]:
dirPath = os.getcwd()+'\\announcements\\muyuan\\notes\\'
fileList = os.listdir(dirPath)

In [3]:
def doc2docx(path):
    w = win32com.client.Dispatch('Word.Application')
    w.Visible = 0
    w.DisplayAlerts = 0
    doc = w.Documents.Open(path)
    newpath = os.path.splitext(path)[0] + '.docx'
    doc.SaveAs(newpath, 12, False, "", True, "", False, False, False, False)
    doc.Close()
    w.Quit()
    os.remove(path)
    return newpath 

In [4]:
# change all doc to docx
for file in tqdm(fileList):
    if file[-3:] == 'DOC':
        doc2docx(dirPath+file)
fileList = os.listdir(dirPath)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 11253.53it/s]


In [5]:
def to_dateTime(dateText):
    dateData = re.findall('(.*?)年(.*?)月(.*?)日',dateText)[0]
    dateTime = datetime.datetime(int(dateData[0]), int(dateData[1]), int(dateData[2]))
    return dateTime

In [6]:
def get_noteData(fullText, noteDate):
    fullText = '\n'+fullText
    noteData = []
    qsData=[]
    for text in re.split('\n(.*?？)\n',fullText)[1:]:
        if re.match('.*?？$',text):
            qsData = [noteDate,text]
        else:
            qsData.append(text)
            noteData.append(qsData)
    return noteData

In [7]:
def read_note_docx(filePath, threshold=100):
    doc = docx.Document(filePath)
    tables = doc.tables
    
    fullText = ''
    for table in tables:
        for i in range(len(table.rows)):
            for j in range(len(table.columns)):
                cellText = table.cell(i,j).text 
                if(len(cellText)>threshold):
                    noteText = cellText
                date = re.match('\d+年\d+月\d+日$',cellText)
                if date:
                    noteDate = to_dateTime(date.group())

    return get_noteData(noteText, noteDate)

In [8]:
def read_note_pdf(filePath):
    with pdfplumber.open(filePath) as pdf:
        fullText = ''
        for pageNum in range(len(pdf.pages)):
            fullText = fullText + pdf.pages[pageNum].extract_text()
        noteDate = to_dateTime(re.findall('日期(.*?)$',fullText,re.S)[0].replace(' ',''))
        noteText = re.findall('人员姓名.*?\n(.*?？.*?)附件清单',fullText,re.S)[0]
        
        return get_noteData(noteText, noteDate)

In [9]:
def read_note_pdf_p5w(filePath):
    with pdfplumber.open(filePath) as pdf:
        fullText = ''
        for pageNum in range(len(pdf.pages)):
            fullText = fullText + pdf.pages[pageNum].extract_text()
        noteDate = to_dateTime(re.findall('日期(.*?)$',fullText,re.S)[0].replace(' ',''))
        
        noteDataRaw = []
        for pageNum in range(1,len(pdf.pages)):
            for table in pdf.pages[pageNum].extract_tables():
                noteDataRaw = noteDataRaw + table[2:]
        
        noteData = []
        i = 0
        while i < len(noteDataRaw):
            if noteDataRaw[i][0] != '':
                noteData.append([noteDate,noteDataRaw[i][1],noteDataRaw[i][2]])
            else:
                noteData.pop()
                noteData.append([noteDate,noteDataRaw[i-1][1]+noteDataRaw[i][1],noteDataRaw[i-1][2]+noteDataRaw[i][2]])
            i+=1
           
        return noteData

In [10]:
def read_note(filePath):
    if 'doc' in filePath[-10:].lower():
        return read_note_docx(filePath)
    elif 'pdf'in filePath[-10:].lower():
        if '路演活动信息' in filePath:
            return read_note_pdf_p5w(filePath)
        else:
            return read_note_pdf(filePath)
    else:
        print('File type error: Should be either docx or pdf.')
        return []

In [11]:
noteData = []
for file in tqdm(fileList):
    filePath = dirPath + file
    try:
        noteData = noteData + read_note(filePath)
    except:
        print(filePath)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.13it/s]


In [27]:
for note in noteData:
    note[1] = note[1].replace('\n','')
    note[2] = note[2].replace('\n','')

tag = ['date', 'question', 'answer']
noteDf = pd.DataFrame(noteData, columns=tag)
noteDf.set_index('date', inplace = True)
noteDf.sort_index(inplace=True)

In [32]:
outputPath = 'muyuan_notes.csv'
noteDf.to_csv(outputPath,sep=',',index=True,header=True, encoding='utf_8_sig') 

[[datetime.datetime(2014, 7, 2, 0, 0),
  '一、公司规模不断扩大，未来的发展规划如何？',
  '1、根据美国、欧洲的经验，养殖企业发展到一定的规模后，往往都覆盖了较长的产业链，很少依赖某一个环节盈利。2、未来3年内公司仍然把主要精力放在养猪上。'],
 [datetime.datetime(2014, 7, 2, 0, 0),
  '二、公司在原粮采购方面的情况如何？',
  '在粮食收获季节集中采购，价格低、运费少，根据近几年的经验以及财务部门测算，公司的集中采购策略由于随用随买的策略。'],
 [datetime.datetime(2014, 7, 2, 0, 0),
  '三、公司新建养殖场的商品猪是否能避开价格低谷时上市？',
  '    1、一个新的养殖场从规划、征地、基础设施建设、投产到商品猪上市，大概需要3年到5年的时间。2、生猪是大宗农产品，价格波动是很正常的，做远期预测难度很大。3、公司作为一个持续经营的主体，很难根据猪价的高低去选择商品猪上市时间。更多的是通过研发、创新等，保持成本领先优势。'],
 [datetime.datetime(2014, 7, 2, 0, 0),
  '四、公司的成本潜力是否已经到了极限，将在哪些方面减少成本？',
  '成本还有进一步下降的空间，公司会在以下几方面做出努力：1、生产管理：提高生产效率；2、育种技术：改良提高生猪生产指标、猪肉品质等；3、疫病防控：注重生猪的疫病防御，减少疫病损失。4、营养配方：降低成本、提高饲料利用率。'],
 [datetime.datetime(2014, 7, 2, 0, 0),
  '五、根据公司披露的信息，新增产能将达到400-500万头，如何解决资金来源？',
  '1、自身造血功能：公司上市前的3年，发展速度也很快，但是负债率始终保持在合理区间，主要是通过经营产生的现金流辅以一定的银行贷款。2、分期投入：新增400-500万头产能，不是一次性投资，将分阶段滚动投入，先期建设的产能盈利后将能够支撑部分或全部的后续资本开支。3、国家政策支持：国家对农业、畜牧业的政策导向是支持的，这一点在银行融资方面也有体现。公司在银行系统里始终保持非常好的信用记录，目前公司的贷款以基准利率为主，基本能够满足建设资金需求。'],
 [datetime